# Import Libraries

In [44]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import random
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import autogluon

%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# Read Dataset

In [45]:
#Read datasets
train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')
X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')
X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')
#Read test datasets
X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')

# Find intervals with broken data in B

In [46]:
# Calculate the difference between consecutive measurements
train_b['diff'] = train_b['pv_measurement'].diff()

# Identify intervals where the difference is zero and the measurement is not 0.0
constant_intervals_B = train_b[(train_b['diff'] == 0) & (train_b['pv_measurement'] != 0) & (train_b['pv_measurement'].notna())]

# Group these intervals and count their lengths
grouped_intervals_B = (constant_intervals_B
                       .assign(group=(constant_intervals_B['time'].diff() != pd.Timedelta(hours=1)).cumsum())
                       .groupby('group')
                       .agg(start_time=('time', 'first'), end_time=('time', 'last'), count=('time', 'count'), constant_value=('pv_measurement', 'first'))
                       .reset_index(drop=True))

# Filter out short intervals (e.g., less than 3 hours) as they might not be considered "broken"
long_constant_intervals_B = grouped_intervals_B[grouped_intervals_B['count'] >= 3]

#long_constant_intervals_B.head(200)  # Display the first few rows of long constant intervals in B


In [47]:
def replace_with_values_from_C(train_b, train_c, constant_intervals):
    train_b_copy = train_b.copy()
    """
    Replace values in dataset B's constant intervals with corresponding values from dataset C.

    :param train_b: DataFrame for dataset B.
    :param train_c: DataFrame for dataset C.
    :param constant_intervals: DataFrame with constant intervals in B to be replaced.
    :return: Modified dataset B with values replaced from C.
    """
    for index, row in constant_intervals.iterrows():
        start_time, end_time = row['start_time'], row['end_time']

        # Iterate through each timestamp within the interval
        for time in pd.date_range(start=start_time, end=end_time, freq='H'):
            # Check if there's a corresponding timestamp in C
            if time in train_c['time'].values:
                # Replace the value in B with the value from C
                value_C = train_c.loc[train_c['time'] == time, 'pv_measurement'].values[0]
                train_b_copy.loc[train_b_copy['time'] == time, 'pv_measurement'] = value_C


    # Replace the constant 0.0 values from line 19140 to 20143 in dataset B with the values from the same timestamp in dataset C.

    # Iterate through each timestamp within the interval
    for time in pd.date_range(start='2021-03-08 15:00:00', end='2021-04-19 11:00:00', freq='H'):
        # Check if there's a corresponding timestamp in C
        if time in train_c['time'].values:
            # Replace the value in B with the value from C
            value_C = train_c.loc[train_c['time'] == time, 'pv_measurement'].values[0]
            train_b_copy.loc[train_b_copy['time'] == time, 'pv_measurement'] = value_C

    return train_b_copy

# Replace values in dataset B using dataset C for the identified constant intervals
train_b_modified = replace_with_values_from_C(train_b, train_c, long_constant_intervals_B)

display(train_b)
display(train_b_modified)

train_b = train_b_modified.drop(columns=['diff'])

# fig, axs = plt.subplots(3, 1, figsize=(20, 10), sharex=True)
# axs[0].plot(train_b_modified['time'], train_b_modified['pv_measurement'])
# axs[0].set_title('Dataset B (Modified)')
# axs[1].plot(train_c['time'], train_c['pv_measurement'])
# axs[1].set_title('Dataset C')
# axs[2].plot(train_b['time'], train_b['pv_measurement'])
# axs[2].set_title('Dataset B')
# plt.show()

,time,pv_measurement,diff
0,2018-12-31 23:00:00,0.000000,NaN
1,2019-01-01 00:00:00,0.000000,0.000000
2,2019-01-01 01:00:00,0.000000,0.000000
3,2019-01-01 02:00:00,0.000000,0.000000
4,2019-01-01 03:00:00,0.000000,0.000000
...,...,...,...
32843,2023-04-30 19:00:00,0.828587,-44.434448
32844,2023-04-30 20:00:00,-0.000000,-0.828587
32845,2023-04-30 21:00:00,-0.000000,0.000000
32846,2023-04-30 22:00:00,-0.000000,0.000000


,time,pv_measurement,diff
0,2018-12-31 23:00:00,0.000000,NaN
1,2019-01-01 00:00:00,0.000000,0.000000
2,2019-01-01 01:00:00,0.000000,0.000000
3,2019-01-01 02:00:00,0.000000,0.000000
4,2019-01-01 03:00:00,0.000000,0.000000
...,...,...,...
32843,2023-04-30 19:00:00,0.828587,-44.434448
32844,2023-04-30 20:00:00,-0.000000,-0.828587
32845,2023-04-30 21:00:00,-0.000000,0.000000
32846,2023-04-30 22:00:00,-0.000000,0.000000


In [48]:
train_b.head()

,time,pv_measurement
0,2018-12-31 23:00:00,0.0
1,2019-01-01 00:00:00,0.0
2,2019-01-01 01:00:00,0.0
3,2019-01-01 02:00:00,0.0
4,2019-01-01 03:00:00,0.0


# Fix metric on ceiling_height_agl:m and cloud_base_agl:m

Found that most likely the **ceiling_height_agl:m** and **cloud_base_agl:m** starts with values in meters, and then suddenly switches into using feet at 26.03.2020. Therefore I transform the values before 26.03.2020 into feet. 

In [49]:
mask_a = X_train_observed_a['date_forecast'] < '2020-03-26'
mask_b = X_train_observed_b['date_forecast'] < '2020-03-26'
mask_c = X_train_observed_c['date_forecast'] < '2020-03-26'

# Apply the conversion from meters to feet (1 meter = 3.28084 feet)
conversion_factor = 3.28084

X_train_observed_a.loc[mask_a, 'ceiling_height_agl:m'] *= conversion_factor
X_train_observed_b.loc[mask_b, 'ceiling_height_agl:m'] *= conversion_factor
X_train_observed_c.loc[mask_c, 'ceiling_height_agl:m'] *= conversion_factor

X_train_observed_a.loc[mask_a, 'cloud_base_agl:m'] *= conversion_factor
X_train_observed_b.loc[mask_b, 'cloud_base_agl:m'] *= conversion_factor
X_train_observed_c.loc[mask_c, 'cloud_base_agl:m'] *= conversion_factor

As the features have a 0.83 correlation, I choose to go forward with ceiling height

# Concat data

In [50]:

#add location to each sample
train_a["location"] = "A"
train_b["location"] = "B"
train_c["location"] = "C"
X_train_estimated_a["location"] = "A"
X_train_estimated_b["location"] = "B"
X_train_estimated_c["location"] = "C"
X_train_observed_a["location"] = "A"
X_train_observed_b["location"] = "B"
X_train_observed_c["location"] = "C"
X_test_estimated_a["location"] = "A"
X_test_estimated_b["location"] = "B"
X_test_estimated_c["location"] = "C"

#copy 23:45 value to 00:00 as it isn't there
def fill_last(frame):
    copy = frame.copy()

    copy['date_forecast'] = pd.to_datetime(copy['date_forecast'])
    
    copy["date_forecast"] = copy["date_forecast"] + pd.Timedelta(minutes=15)
    
    copy = copy[copy["date_forecast"].apply(lambda time : time.hour == 00 and time.minute == 00)]

    copy = copy[frame.columns]
    
    frame = pd.concat([
        frame,
        copy.astype(frame.dtypes)
    ],)

    frame = frame.drop_duplicates(subset="date_forecast", keep="first")

    return frame

#fill last
X_test_estimated_a = fill_last(X_test_estimated_a.copy())
X_test_estimated_b = fill_last(X_test_estimated_b.copy())
X_test_estimated_c = fill_last(X_test_estimated_c.copy())

#remove extra minute 00 sample
X_train_observed_a = X_train_observed_a.iloc[:-1,:]
X_train_observed_b = X_train_observed_b.iloc[:-1,:]
X_train_observed_c = X_train_observed_c.iloc[:-1,:]

#add date_calc column same as date_forecast column to observed data
X_train_observed_a.insert(0, "date_calc", X_train_observed_a["date_forecast"])
X_train_observed_b.insert(0, "date_calc", X_train_observed_b["date_forecast"])
X_train_observed_c.insert(0, "date_calc", X_train_observed_c["date_forecast"])

#concat all the samples and remove date_calc column
X_train_raw = pd.concat([X_train_observed_a,
                     X_train_observed_b,
                     X_train_observed_c,
                     X_train_estimated_a,
                     X_train_estimated_b,
                     X_train_estimated_c,
                     X_test_estimated_a,
                     X_test_estimated_b,
                     X_test_estimated_c])


## Clean and preprocess data

In [51]:

#map snow density to one and zero
X_train_raw["snow_density:kgm3"] = X_train_raw["snow_density:kgm3"].apply(lambda a : np.isnan(a)).map({True: 0, False: 1})

#fix ceiling_height NaN values to -666 because the docs hints to it
#also rename the features with their proper metric and remove the old ones
X_train_raw["ceiling_height_agl:ft"] = X_train_raw["ceiling_height_agl:m"].fillna(-666)
X_train_raw["cloud_base_agl:ft"] = X_train_raw["cloud_base_agl:m"].fillna(-666)
X_train_raw.drop(columns=["ceiling_height_agl:m", "cloud_base_agl:m"], inplace=True)

# #categorizing ceiling_height_agl:ft
# #found categories on google
# X_train_raw['ceiling_height_agl:ft'] = pd.cut(X_train_raw['ceiling_height_agl:ft'], bins=[float('-inf'), 0, 500, 1000, 3000, 5000, 12000, float('inf')], labels=[-666, 1, 2, 3, 4, 5, 6])

#decided to drop cloud_base_agl:ft because of high correlation to ceiling_height_agl:ft
X_train_raw.drop(columns=["cloud_base_agl:ft"], inplace=True)

# #categorizing dew_or_rime:idx
# dew_or_rime_categories = {
#     -1.0: "Rime",
#     0.0: "None",
#     1.0: "Dew"
# }
# X_train_raw['dew_or_rime:idx'] = X_train_raw['dew_or_rime:idx'].map(dew_or_rime_categories)

#casting floats to int for categorical features
X_train_raw['is_day:idx'] = X_train_raw['is_day:idx'].astype(int)
X_train_raw['dew_or:rime:idx'] = X_train_raw['dew_or_rime:idx'].astype(int)

#remove some weird artifacts from train_b target values
train_b = pd.concat([train_b[:18690], train_b[20142:]])
train_b["rolling"] = train_b["pv_measurement"].rolling(4).mean()
train_b["keep"] = train_b["pv_measurement"] - train_b["rolling"] != 0 + train_b["pv_measurement"].apply(lambda a: a==0)
train_b = train_b[train_b["keep"]]
train_b = train_b.iloc[:,:3]

parse_dates = ['time']
X_test_targets = pd.read_csv("test.csv", parse_dates=parse_dates)

train_a["id"] = -10
train_b["id"] = -10
train_c["id"] = -10

X_test_targets = X_test_targets.rename(columns = {"prediction" : "pv_measurement"})

targets = pd.concat([train_a,
                     train_b.astype(train_a.dtypes),
                     train_c.astype(train_a.dtypes),
                     X_test_targets.astype(train_a.dtypes)]).dropna()


## Split into timeframes

In [52]:

features00 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 0)].copy()
features00["merge_time"] = features00["date_forecast"]


features15 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 15)].copy()
features15["merge_time"] = features15["date_forecast"] + pd.Timedelta(minutes=-15)


features30 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 30)].copy()
features30["merge_time"] = features30["date_forecast"] + pd.Timedelta(minutes=-30)


features45 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 45)].copy()
features45["merge_time"] = features45["date_forecast"] + pd.Timedelta(minutes=-45)


X_train_raw["date_forecast"] = X_train_raw["date_forecast"] + pd.Timedelta(minutes = -60)
features60 = X_train_raw[X_train_raw["date_forecast"].apply(lambda time: time.minute == 00)].copy()
features60["merge_time"] = features60["date_forecast"]

dataset = targets
dataset = dataset.rename(columns={"time": "merge_time"})

display(X_train_raw)

,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,effective_cloud_cover:p,elevation:m,fresh_snow_12h:cm,fresh_snow_1h:cm,fresh_snow_24h:cm,fresh_snow_3h:cm,fresh_snow_6h:cm,is_day:idx,is_in_shadow:idx,msl_pressure:hPa,precip_5min:mm,precip_type_5min:idx,pressure_100m:hPa,pressure_50m:hPa,prob_rime:p,rain_water:kgm2,relative_humidity_1000hPa:p,sfc_pressure:hPa,snow_density:kgm3,snow_depth:cm,snow_drift:idx,snow_melt_10min:mm,snow_water:kgm2,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,location,ceiling_height_agl:ft,dew_or:rime:idx
0,2019-06-02 22:00:00,2019-06-02 21:00:00,7.7,1.230,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,98.699997,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.799988,0.0,0.0,994.200012,1000.299988,0.0,0.0,73.099998,1006.299988,0,0.0,0.0,-0.0,0.1,342.834015,-3.202,0.0,285.899994,100.000000,39640.101562,3.7,-3.6,-0.8,-0.0,A,5724.737793,0
1,2019-06-02 22:15:00,2019-06-02 21:15:00,7.7,1.229,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,99.000000,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.500000,0.0,0.0,993.900024,999.900024,0.0,0.0,72.199997,1006.000000,0,0.0,0.0,-0.0,0.2,346.294006,-3.650,0.0,286.100006,100.000000,40123.898438,3.6,-3.6,-0.6,-0.0,A,5688.976562,0
2,2019-06-02 22:30:00,2019-06-02 21:30:00,7.7,1.228,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,99.199997,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1006.099976,0.0,0.0,993.599976,999.599976,0.0,0.0,71.199997,1005.599976,0,0.0,0.0,-0.0,0.2,349.768005,-3.998,0.0,286.299988,100.000000,40628.300781,3.6,-3.6,-0.4,-0.0,A,5654.527832,0
3,2019-06-02 22:45:00,2019-06-02 21:45:00,7.7,1.226,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,99.400002,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1005.799988,0.0,0.0,993.299988,999.299988,0.0,0.0,70.199997,1005.299988,0,0.0,0.0,-0.0,0.2,353.251007,-4.247,0.0,286.600006,100.000000,41153.601562,3.5,-3.5,-0.2,-0.0,A,5621.391113,0
4,2019-06-02 23:00:00,2019-06-02 22:00:00,7.7,1.225,0.0,0.0,0.0,280.299988,0.0,0.0,0.0,0.0,99.599998,6.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1005.500000,0.0,0.0,993.000000,999.000000,0.0,0.0,69.199997,1005.000000,0,0.0,0.0,-0.0,0.2,356.742004,-4.393,0.0,286.799988,100.000000,41699.898438,3.5,-3.5,0.0,-0.0,A,5589.238770,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2207,2023-06-18 07:00:05,2023-06-19 23:00:00,10.7,1.193,0.0,0.0,0.0,285.600006,0.0,0.0,0.0,0.0,27.799999,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1011.000000,0.0,0.0,995.299988,1001.200012,0.0,0.0,59.200001,1007.000000,0,0.0,0.0,-0.0,0.0,6.356000,-3.011,0.0,293.700012,43.500000,46996.800781,4.0,-2.0,3.5,-0.0,C,11394.299805,0
2303,2023-06-21 07:00:30,2023-06-22 23:00:00,9.4,1.228,0.0,0.0,0.0,283.299988,0.0,0.0,0.0,0.0,95.599998,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1013.900024,0.0,0.0,997.799988,1003.799988,0.0,0.0,83.199997,1009.900024,0,0.0,0.0,-0.0,0.0,6.206000,-2.996,0.0,285.399994,96.800003,33542.898438,1.3,-0.6,1.1,-0.0,C,1541.400024,0
2399,2023-06-25 07:01:23,2023-06-26 23:00:00,9.8,1.187,0.0,0.0,0.0,284.200012,0.0,0.0,0.0,0.0,2.000000,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1004.799988,0.0,0.0,989.200012,995.000000,0.0,0.0,53.099998,1000.799988,0,0.0,0.0,-0.0,0.0,6.015000,-3.071,0.0,295.500000,2.000000,48980.699219,3.1,-2.0,2.5,-0.0,C,-666.000000,0
2687,2023-06-29 07:00:05,2023-06-30 23:00:00,9.3,1.220,0.0,0.0,0.0,283.100006,0.0,0.0,0.0,0.0,100.000000,24.0,0.0,0.0,0.0,0.0,0.0,0,1.0,1004.000000,0.0,0.0,989.200012,995.200012,0.0,0.1,88.800003,1001.200012,0,0.0,0.0,-0.0,0.2,5.837000,-3.254,0.1,284.399994,100.000000,9935.700195,1.8,1.6,-0.9,0.0,C,2006.300049,0


# Define helper functions

In [53]:
#averages the features meassured at target time +00, +15, +30, +45 and +60
def add_feature_average_00_60(dataset, f00, f15, f30, f45, f60, column_name, categorical=False):
    dataset = pd.merge(
        left=dataset,
        right = f00[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner")
    dataset = pd.merge(
        left=dataset,
        right = f15[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_15"])
    dataset = pd.merge(
        left=dataset,
        right = f30[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_30"])
    dataset = pd.merge(
        left=dataset,
        right = f45[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_45"])
    dataset = pd.merge(
        left=dataset,
        right = f60[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner",
        suffixes=["", "_60"])


    dataset[column_name] = (dataset[column_name] +
                            dataset[column_name + "_15"] +
                            dataset[column_name + "_30"] +
                            dataset[column_name + "_45"] +
                            dataset[column_name + "_60"])/5
    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)
    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

#adds a single feature from one observation
def add_feature(dataset, f, column_name, count=True, categorical=False):
    if count:
        if categorical:
            add_to_categorical_features(column_name)
        else:
            add_to_numerical_features(column_name)
    return pd.merge(
            left=dataset,
            right=f[["location", "merge_time", column_name]],
            on=["location", "merge_time"],
            how="inner"
  )

#adds an One Hot Encoding of the column to the dataset
def OHE(dataset, f, column_name, suffix=""):

    dataset = pd.merge(
        left=dataset,
        right = f[["location", "merge_time", column_name]],
        on=["location", "merge_time"],
        how="inner")

    values = dataset[column_name].unique()

    for value in values:
        dataset[column_name + "_" + suffix + str(value)] = dataset[column_name].apply(lambda a : a == value).map({True: 1, False: 0})

    dataset = dataset.drop([column_name], axis=1)
    return dataset

def OHE_all(dataset, f00, f15, f30, f45, f60, column_name):
    dataset = OHE(dataset, f00, column_name, suffix="00_")
    dataset = OHE(dataset, f15, column_name, suffix="15_")
    dataset = OHE(dataset, f30, column_name, suffix="30_")
    dataset = OHE(dataset, f45, column_name, suffix="45_")
    dataset = OHE(dataset, f60, column_name, suffix="60_")

    return dataset

#adds all observations
def add_all(dataset, f00, f15, f30, f45, f60, column_name, categorical=False):
    dataset[column_name + "_00"] = add_feature(dataset, f00, column_name, count=False,categorical=categorical)[column_name]
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False, categorical=categorical)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False, categorical=categorical)[column_name]

    feature_names = [column_name + "_00", column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]
    for feat in feature_names:
        if categorical:
            add_to_categorical_features(feat)
        else:
            add_to_numerical_features(feat)

    return dataset

#finds mode (typetall) for a row
def find_mode_with_priority(row, priority_list, max=False):
    # Check for prioritized values in the row and set it as the mode if it is
    for value in priority_list:
        if value in row.values:
            return value

    # Calculate the mode for the row
    if max:
        mode_value = row.mode().max()
    else:
        mode_value = row.mode().min()

    return mode_value

def add_most_frequent_feature(dataset, f15, f30, f45, f60, column_name, priority_list, max=False, categorical=True):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False)[column_name]

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    dataset[column_name] = dataset[feature_names].apply(find_mode_with_priority, args=(priority_list, max), axis=1)

    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)

    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

def add_accumulated(dataset, f15, f30, f45, f60, column_name, time_interval, categorical=False):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name, count=False)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name, count=False)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name, count=False)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name, count=False)[column_name]

    time_multiplier = 15/time_interval

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    dataset[column_name] = dataset[feature_names].sum(axis=1)*time_multiplier

    dataset = dataset.drop([column_name + "_15",
                            column_name + "_30",
                            column_name + "_45",
                            column_name + "_60"],
                           axis=1)
    if categorical:
        add_to_categorical_features(column_name)
    else:
        add_to_numerical_features(column_name)

    return dataset

def add_accumulated_all(dataset, f15, f30, f45, f60, column_name, time_interval, categorical=False):
    dataset[column_name + "_15"] = add_feature(dataset, f15, column_name)[column_name]
    dataset[column_name + "_30"] = add_feature(dataset, f30, column_name)[column_name]
    dataset[column_name + "_45"] = add_feature(dataset, f45, column_name)[column_name]
    dataset[column_name + "_60"] = add_feature(dataset, f60, column_name)[column_name]

    time_multiplier = 15/time_interval

    feature_names = [column_name + "_15", column_name + "_30", column_name + "_45", column_name + "_60"]

    for feat in feature_names:
        dataset[feat] = dataset[feat]*time_multiplier
    if categorical:
        add_to_categorical_features(column_name)    
    else:
        add_to_numerical_features(column_name)

    return dataset

numerical_feature_names = []
categorical_feature_names = []

def add_to_numerical_features(feature_name):
    numerical_feature_names.append(feature_name)

def add_to_categorical_features(feature_name):
    categorical_feature_names.append(feature_name)


# Feature info

### Features removed because of correlations:

- **fresh_snow_12h:cm**: fresh_snow_24h:cm = **0.82**

- **fresh_snow_3h:cm**: fresh_snow_1h:cm = **0.81**

- **fresh_snow_6h:cm**: fresh_snow_24h:cm = **0.83**

- **diffuse_rad:W**: diffuse_rad_1h:J = **0.99**

- **direct_rad:W**: direct_rad_1h:J = **0.99**

- **pressure_100m:hPa**: msl_pressure:hPa = **1.00**

- **pressure_50m:hPa**: msl_pressure:hPa = **1.00**

- **sfc_pressure:hPa**: msl_pressure:hPa = **1.00**

- **absolute_humidity_2m:gm3**: t1000:hPa = **0.90**

- **air_density_2m:kgm3**: t1000:hPa = **0.90**

- **dew_point_2m:K**: t1000:hPa = **0.91**

- **clear_sky_rad:W**: sun_elevation = **0.83**

- **clear_sky_energy_1h:J**: sun_elevation = **0.82**

- **total_cloud_cover:p**: effective_cloud_cover:p = **0.94**

- **cloud_base_agl:m**: ceiling_height_agl:m = **0.83** (after recalculating to feet)

- **elevation:m**: highly correlated to location

- **is_in_shadow:idx**: highly correlated to is_day:idx

### High correlations still in the set:

- **is_day:idx**: sun_elevation = **0.81**

- **diffuse_rad_1h:J**: sun_elevation = **0.80**

### Other features removed:

- **snow_drift:idx**: Almost exclusively 0, so doesn't add much data

- **wind_speed_w_1000hPa:ms**: In relation to the dummy data found in the docs this data is really weird (binary as opposed to continous values)


# Add numerical features

In [54]:
# SNOW AND PRECIPITATION

# tar verdi fra +60 siden den viser måling mellom 00 og 60, #!kan det være gunstig å ha med 3h, 6h, 12h????
# dataset = add_feature(dataset, features60, "fresh_snow_1h:cm")
# dataset = add_feature(dataset, features60, "fresh_snow_24h:cm")

# bruker bare 24h fordi den har høyest korrelasjon med pv_measurement
dataset = add_feature(dataset, features60, "fresh_snow_24h:cm")

# tar alle akkumulerte verdier og ganger med 3 for å få en bedre verdi (ikke helt etter boka menmen...)
dataset = add_all(dataset, features00, features15, features30, features45, features60, "precip_5min:mm")
# disse tar jeg bare gjennomsnittet av
#dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "snow_depth:cm")
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "snow_water:kgm2")
# tar akkumulert verdi og ganger med 3/2 for å få en bedre verdi (ikke helt etter boka menmen...)
#dataset = add_accumulated(dataset, features15, features30, features45, features60, "snow_melt_10min:mm", 10)


# ACCUMULATIVE FEATURES

# tar verdi fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "diffuse_rad_1h:J")
# tar verdi fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "direct_rad_1h:J")#!Try without

# PRESSURE

# tar gjennomsnittet da dette er punktmålinger ##kan hende denne burde kjøres per kvarter
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "msl_pressure:hPa")


# TEMPERATURE

# gjennomsnitt siden variasjonen hvert kvarter sannsynligvis er lav? ##kan hende denne burde kjøres per kvarter
# update: kjører hvert kvarter
dataset = add_all(dataset, features00, features15, features30, features45, features60, "t_1000hPa:K")


# SUN

#legger til alle siden har testing har vist at disse er svært viktige
dataset = add_all(dataset, features00, features15, features30, features45, features60, "sun_azimuth:d")
dataset = add_all(dataset, features00, features15, features30, features45, features60, "sun_elevation:d")

#tar verdien fra +60 siden den viser måling mellom 00 og 60
dataset = add_feature(dataset, features60, "clear_sky_energy_1h:J")


# DAY AND SHADOW

#gjennomsnitt fordi jeg vet ikke
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "visibility:m")


# CLOUDS

#gjennomsnitt fordi verdien er trolig momentan
#SANNSYNLIGVIS VELDIG VIKTIG
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "effective_cloud_cover:p")
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "ceiling_height_agl:ft")


# HUMIDITY AND RIME

#tar gjennomsnitt fordi jeg vet ikke #!diskuter
#dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "prob_rime:p")
#tar gjennomsnitt fordi jeg vet ikke #!diskuter
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "relative_humidity_1000hPa:p")


# WIND

# Gjennomsnitt fordi lite variabel #! try without
# dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_u_10m:ms")
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_v_10m:ms")
# dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "wind_speed_10m:ms")


# OTHERS (Up for discussion)
# Gjennomsnitt fordi?
dataset = add_feature_average_00_60(dataset, features00, features15, features30, features45, features60, "super_cooled_liquid_water:kgm2")

# Standardize data

In [55]:
# make trig transform on solar angles
def apply_trig(dataset, feature_name, suffixes):
    for suffix in suffixes:
        column_name = f"{feature_name}_{suffix}"
        if column_name in dataset:
            dataset[column_name] = dataset[column_name].apply(lambda d: np.cos((d * np.pi) / 180))

# apply trig transform on solar angles
suffixes = ['00', '15', '30', '45', '60']
apply_trig(dataset, 'sun_azimuth:d', suffixes)
apply_trig(dataset, 'sun_elevation:d', suffixes)

# # all features except pv_measurement, merge_time, location and id
# columns_to_exclude = ['merge_time', 'pv_measurement', 'location', 'id']
# # standardize the features
# scaler = StandardScaler()
# scaled_columns = scaler.fit_transform(dataset.drop(columns=columns_to_exclude))
# scaled_dataset = pd.DataFrame(scaled_columns, columns=dataset.drop(columns=columns_to_exclude).columns)

# # add the excluded columns back to the dataset
# dataset = pd.concat([dataset[columns_to_exclude], scaled_dataset], axis=1)


## Trying first without standardization


def standardize(dataset, feature_name, minus_min=False):
    if minus_min:
        dataset[feature_name] = (dataset[feature_name] - dataset[feature_name].min())/dataset[feature_name].std()
    else:
        dataset[feature_name] = dataset[feature_name]/dataset[feature_name].std()
    
    return dataset

def standardize_all(dataset, feature_name, accumulated=False, minus_min=False):
    if not accumulated:
        dataset[feature_name + "_00"] = standardize(dataset, feature_name + "_00")[feature_name + "_00"]

    dataset[feature_name + "_15"] = standardize(dataset, feature_name + "_15", minus_min=minus_min)[feature_name + "_15"]
    dataset[feature_name + "_30"] = standardize(dataset, feature_name + "_30", minus_min=minus_min)[feature_name + "_30"]
    dataset[feature_name + "_45"] = standardize(dataset, feature_name + "_45", minus_min=minus_min)[feature_name + "_45"]
    dataset[feature_name + "_60"] = standardize(dataset, feature_name + "_60", minus_min=minus_min)[feature_name + "_60"]

    return dataset

# standardize the features
dataset = standardize(dataset, 'fresh_snow_24h:cm')
dataset = standardize_all(dataset, 'precip_5min:mm')
#dataset = standardize(dataset, 'snow_depth:cm')
dataset = standardize(dataset, 'snow_water:kgm2')
#dataset = standardize(dataset, 'snow_melt_10min:mm')
dataset = standardize(dataset, 'diffuse_rad_1h:J')
dataset = standardize(dataset, 'direct_rad_1h:J')
dataset = standardize(dataset, 'msl_pressure:hPa', minus_min=True)
dataset = standardize_all(dataset, 't_1000hPa:K', minus_min=True)
dataset = standardize_all(dataset, 'sun_azimuth:d')
dataset = standardize_all(dataset, 'sun_elevation:d')
dataset = standardize(dataset, 'clear_sky_energy_1h:J')
dataset = standardize(dataset, 'visibility:m')
dataset = standardize(dataset, 'effective_cloud_cover:p')
dataset = standardize(dataset, 'ceiling_height_agl:ft')
# dataset = standardize(dataset, 'prob_rime:p')
dataset = standardize(dataset, 'relative_humidity_1000hPa:p')
# dataset = standardize(dataset, 'wind_speed_u_10m:ms')
dataset = standardize(dataset, 'wind_speed_v_10m:ms')
# dataset = standardize(dataset, 'wind_speed_10m:ms')
dataset = standardize(dataset, 'super_cooled_liquid_water:kgm2')


# Add categorical features

In [56]:
# reason I'm adding it here is to prevent the massive -666 value getting in the way of the scaling
# add most frequent bc don't wanna have 100 000 features, also don't add priority list and pick the min value if conflict
# dataset = add_most_frequent_feature(dataset, features00, features15, features30, features45, "ceiling_height_agl:ft", priority_list=[], max=False)
#! test this aswell
# dataset = add_all(dataset, features00, features15, features30, features45, features60, "ceiling_height_agl:ft")

# add most frequent bc don't wanna have 100 000 features
dataset = add_most_frequent_feature(dataset, features15, features30, features45, features60, "precip_type_5min:idx", priority_list=[3,4,2], max=True)
#! test this aswell
# dataset = add_all(dataset, features00, features15, features30, features45, features60, "precip_type_5min:idx")

# # categorizing ceiling_height_agl:ft
# # found categories on google
# labels=["VFR1", "LIFR", "IFR", "MVFR", "VFR4", "VFR3", "VFR2"]
# ceiling_height_agl_ft_categories = {
#     -666: "VFR1",
#     1: "LIFR",
#     2: "IFR",
#     3: "MVFR",
#     4: "VFR4",
#     5: "VFR3",
#     6: "VFR2",
# }
# # map the values to their labels
# dataset['ceiling_height_agl:ft'] = dataset['ceiling_height_agl:ft'].map(ceiling_height_agl_ft_categories)

# #categorizing precip_type_5min:idx
# #found this in the docs
# precip_types = {
#     0: "None",
#     1: "Rain",
#     2: "Rain_and_snow_mixed",
#     3: "Snow",
#     4: "Sleet",
#     5: "Freezing_rain",
#     6: "Hail",
# }
# #map the values to their labels
# dataset['precip_type_5min:idx'] = dataset['precip_type_5min:idx'].map(precip_types)
    
# Add feature from 60 because pretty consistent
dataset = add_feature(dataset, features60, "snow_density:kgm3", categorical=True)

# OHE av kategorisk variabel #!Opp til diskusjon om man skal ta gjennomsnitt eller flere av målingene
dataset = OHE(dataset, features00, features15, features30, features45, features60, "dew_or_rime:idx")
# Husk at denne er mappet til strings lenger opp

# tar alle verdiene siden disse nok er ekstremt viktige for modellen og gir ikke mening å standardisere
dataset = add_all(dataset, features00, features15, features30, features45, features60, "is_day:idx", categorical=True)

#mapper location til tall og legger til som kategorisk variabel
location_mapping = {
    "A": 1,
    "B": 2,
    "C": 3
}
dataset["location"] = dataset["location"].map(location_mapping)
add_to_categorical_features("location")

#adding feature for summer months
dataset['merge_time'] = pd.to_datetime(dataset['merge_time'])
dataset['is_summer_month'] = dataset['merge_time'].dt.month.isin([5, 6, 7]).astype(int)

# 4 precip_5min:mm values are NaN, set them to 0 because precip_type is 0.0 (None)
# 3 precip_type:idx values are NaN, set them to 0.0 (None)
dataset = dataset.fillna(0.0)

In [57]:
display(dataset.head())
print(dataset.drop(columns=['merge_time', 'pv_measurement']).shape)
print(len(numerical_feature_names))
print(len(categorical_feature_names))
print(len(numerical_feature_names) + len(categorical_feature_names))

,merge_time,pv_measurement,location,id,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,ceiling_height_agl:ft,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60,is_summer_month
0,2019-06-02 22:00:00,0.00,1,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.765305,0.000000,0.000000,4.735564,43.430286,4.270978,4.302045,4.346641,4.389761,1.388385,1.411783,1.430145,1.443321,1.450693,10.254351,10.250209,10.247080,10.245227,10.240512,0.000000,2.277807,2.889253,1.895693,4.986034,-0.214018,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1
1,2019-06-02 23:00:00,0.00,1,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.850339,0.000000,0.000000,4.654308,43.567001,4.392570,4.393256,4.407432,4.420142,1.450767,1.453151,1.450181,1.441875,1.432093,10.240212,10.240247,10.242091,10.245598,10.246780,0.000000,1.743335,2.907314,1.853466,4.735262,0.192617,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2,2019-06-03 00:00:00,0.00,1,-10,0.0,0.0,0.0,0.0,0.0,0.0,1.955779,0.000000,0.000000,4.609854,43.597385,4.407771,4.393256,4.392233,4.404952,1.432166,1.414699,1.392173,1.364742,1.332111,10.246479,10.251056,10.256636,10.262772,10.264517,0.000000,0.633054,2.913141,1.680576,4.803909,0.406635,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1
3,2019-06-03 01:00:00,0.00,1,-10,0.0,0.0,0.0,0.0,0.0,0.0,2.125847,0.035346,0.000000,4.563864,43.582195,4.377370,4.362855,4.346641,4.344195,1.332179,1.295376,1.254300,1.209184,1.159840,10.264216,10.268524,10.271742,10.273151,10.267616,0.007823,0.134138,2.913141,1.372868,5.239610,0.481541,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1,1
4,2019-06-03 02:00:00,19.36,1,-10,0.0,0.0,0.0,0.0,0.0,0.0,0.935373,0.274512,0.007691,4.494875,43.521431,4.331774,4.317248,4.316243,4.329004,1.159899,1.107313,1.051410,0.992435,0.930242,10.267315,10.261632,10.252054,10.237811,10.213802,0.122154,0.819880,2.320608,1.065049,5.633280,0.492242,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1


(93533, 47)
32
13
45


In [58]:
dataset.isna().sum()

merge_time                        0
pv_measurement                    0
location                          0
id                                0
fresh_snow_24h:cm                 0
precip_5min:mm_00                 0
precip_5min:mm_15                 0
precip_5min:mm_30                 0
precip_5min:mm_45                 0
precip_5min:mm_60                 0
snow_water:kgm2                   0
diffuse_rad_1h:J                  0
direct_rad_1h:J                   0
msl_pressure:hPa                  0
t_1000hPa:K_00                    0
t_1000hPa:K_15                    0
t_1000hPa:K_30                    0
t_1000hPa:K_45                    0
t_1000hPa:K_60                    0
sun_azimuth:d_00                  0
sun_azimuth:d_15                  0
sun_azimuth:d_30                  0
sun_azimuth:d_45                  0
sun_azimuth:d_60                  0
sun_elevation:d_00                0
sun_elevation:d_15                0
sun_elevation:d_30                0
sun_elevation:d_45          

# Split into training set and test set

In [59]:
testset = dataset[dataset["id"].apply(lambda id: id != -10)]
testset = testset.drop(columns=["id", "pv_measurement"])

In [60]:
from sklearn.model_selection import train_test_split

# extract real dataset
dataset = dataset[dataset["id"].apply(lambda id: id == -10)]
dataset = dataset.drop(columns=["id", "merge_time"], axis=1)

# dataset = dataset.sort_values(by="merge_time")

# randomly split dataset into train and eval
trainset, evalset = train_test_split(dataset, test_size=0.2, random_state=42)

# split into X and Y
trainsetX = trainset.drop(columns="pv_measurement", axis=1)
trainsetY = trainset["pv_measurement"]
evalsetX = evalset.drop(columns="pv_measurement", axis=1)
evalsetY = evalset["pv_measurement"]

display(trainsetX)
display(evalsetX)
display(trainsetY)
display(evalsetY)

,location,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,ceiling_height_agl:ft,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60,is_summer_month
16516,1,0.000000,0.0,0.0,0.0,0.0,0.0,1.530610,0.000000,0.000000,4.850542,41.956791,2.766258,2.766689,2.766045,2.779679,1.122485,1.063312,1.000724,0.935038,0.866258,10.151398,10.180272,10.207259,10.231384,10.247303,0.000000,0.188527,2.909062,0.276784,6.536904,-0.214018,0.957127,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
58757,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.910787,42.063122,2.857449,2.857895,2.857230,2.870812,0.558660,0.468785,0.379192,0.290157,0.201797,8.267420,8.432013,8.594129,8.752418,8.901926,0.000000,2.566034,2.822833,1.797427,5.877048,1.754951,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
45947,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.425169,3.156969,1.348459,4.603725,42.047932,2.872649,2.888300,2.902827,2.931574,-0.888118,-0.803809,-0.717780,-0.630574,-0.542484,8.171122,8.317660,8.466503,8.616030,8.760757,2.520577,0.567314,2.655619,0.978962,4.288354,0.856074,0.574276,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1
29556,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.322890,1.430842,3.985909,42.716328,3.556614,3.572371,3.601938,3.645477,-1.391103,-1.416249,-1.435110,-1.447473,-1.452683,9.837360,9.815462,9.799400,9.789395,9.781649,1.121383,2.922486,0.999207,2.556814,5.007050,-0.374532,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0
25171,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.662096,0.429375,6.760716,42.746704,3.541414,3.511566,3.495549,3.493582,0.012325,0.097269,0.181588,0.265134,0.347634,10.081620,10.135400,10.180701,10.217402,10.241181,0.346709,2.202336,0.067585,-0.223229,4.034780,-0.877475,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.538259,42.032742,2.842254,2.842697,2.842036,2.855627,1.434858,1.448863,1.453238,1.447899,1.437328,8.070942,8.049572,8.043585,8.053342,8.106307,0.000000,0.926579,2.846721,0.236692,6.308546,1.230606,1.148552,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
54886,2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.494315,0.165898,5.859282,43.202427,4.012590,3.998013,3.981889,3.994838,0.096101,0.179563,0.261767,0.342617,0.421893,9.595279,9.698728,9.793684,9.879849,9.952944,1.096305,0.658430,2.861870,0.291411,5.453957,-0.813270,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1
76820,3,1.812876,0.0,0.0,0.0,0.0,0.0,0.765305,0.144136,0.000056,2.179981,42.579605,3.404622,3.420355,3.434758,3.463206,-1.449180,-1.453035,-1.451644,-1.445014,-1.432672,10.254301,10.254218,10.255536,10.258190,10.257615,0.105709,3.385048,2.895662,1.628719,4.031977,2.835744,0.382851,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0
860,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.967334,0.294011,5.554208,43.111282,3.921394,3.937208,3.936292,3.949267,0.465768,0.542036,0.616539,0.689148,0.759471,9.876274,9.950718,10.016611,10.074178,10.119409,0.709215,1.716586,2.347991,0.456252,6.181058,-0.663457,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,1


,location,fresh_snow_24h:cm,precip_5min:mm_00,precip_5min:mm_15,precip_5min:mm_30,precip_5min:mm_45,precip_5min:mm_60,snow_water:kgm2,diffuse_rad_1h:J,direct_rad_1h:J,msl_pressure:hPa,t_1000hPa:K_00,t_1000hPa:K_15,t_1000hPa:K_30,t_1000hPa:K_45,t_1000hPa:K_60,sun_azimuth:d_00,sun_azimuth:d_15,sun_azimuth:d_30,sun_azimuth:d_45,sun_azimuth:d_60,sun_elevation:d_00,sun_elevation:d_15,sun_elevation:d_30,sun_elevation:d_45,sun_elevation:d_60,clear_sky_energy_1h:J,visibility:m,effective_cloud_cover:p,ceiling_height_agl:ft,relative_humidity_1000hPa:p,wind_speed_v_10m:ms,super_cooled_liquid_water:kgm2,precip_type_5min:idx,snow_density:kgm3,dew_or_rime:idx_00,dew_or_rime:idx_15,dew_or_rime:idx_30,dew_or_rime:idx_45,dew_or_rime:idx_60,is_day:idx_00,is_day:idx_15,is_day:idx_30,is_day:idx_45,is_day:idx_60,is_summer_month
37181,2,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,6.788310,41.850452,2.659862,2.660276,2.644458,2.658161,1.451103,1.453083,1.449326,1.439859,1.428867,10.157255,10.156976,10.159882,10.165832,10.174391,0.000000,2.030349,2.282737,1.387647,4.887968,-0.074906,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1
78736,3,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.000000,1.110706,0.768991,6.283942,41.030155,1.869501,1.915400,1.945347,1.989824,-0.522431,-0.603586,-0.683382,-0.761535,-0.837394,10.151479,10.106198,10.055404,9.999992,9.936548,0.654073,2.642704,1.961126,2.506946,4.450866,1.359017,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1,0
64242,2,1.89528,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,5.204670,41.242825,2.036693,2.021808,2.006143,2.020205,1.441555,1.451240,1.452858,1.446381,1.435805,9.007940,8.995570,8.994485,9.004496,9.049557,0.000000,1.721139,1.820130,1.307169,5.930285,1.326914,0.000000,0.0,1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
3431,1,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.595237,0.000000,0.000000,4.427416,43.050518,3.860598,3.861200,3.845103,3.858129,1.170010,1.231410,1.286846,1.335544,1.376238,8.556306,8.461893,8.376793,8.302207,8.235965,0.000000,1.062086,2.869444,0.960889,6.265116,0.032103,0.191425,0.0,0,1.0,1.0,1.0,1.0,1.0,0,0,0,0,0,0
30125,1,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,5.411632,41.956791,2.766258,2.766689,2.750846,2.764489,1.159440,1.087329,1.010684,0.930408,0.847003,7.699423,7.824929,7.959156,8.100103,8.242279,0.000000,3.233562,0.040201,-0.223229,3.712558,0.481541,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89403,3,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,3.930717,41.911213,2.705457,2.705879,2.690055,2.688541,0.283290,0.372014,0.461270,0.550748,0.639910,8.872013,8.720867,8.565542,8.407271,8.243883,0.000000,2.909558,1.514833,2.717190,5.022460,0.834672,0.000000,0.0,0,-1.0,-1.0,-1.0,-1.0,-1.0,0,0,0,0,0,0
63802,2,0.00000,0.0,0.0,0.00000,0.000000,0.0,2.295915,0.000000,0.000000,4.714099,41.880836,2.705457,2.705879,2.720448,2.749298,1.042514,1.112568,1.178149,1.238505,1.292405,8.835495,8.731362,8.633199,8.542649,8.457330,0.000000,0.324645,2.913141,0.143832,6.323956,1.401821,1.531403,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
44924,2,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.425169,0.000000,0.000000,3.046149,41.744114,2.538270,2.508261,2.492478,2.475889,1.447535,1.452943,1.451228,1.442407,1.430728,9.559421,9.554189,9.557124,9.568085,9.602383,0.000000,1.773016,2.784962,0.943514,5.283039,-2.193688,0.957127,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0
43417,2,0.00000,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,3.007823,41.759308,2.568665,2.569066,2.568468,2.582213,0.667980,0.580889,0.493252,0.405435,0.317565,8.693880,8.836244,8.976589,9.113644,9.242206,0.000000,3.090186,0.040784,1.964070,4.561543,1.177101,0.000000,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0


16516       0.0000
58757       0.0000
45947     347.5875
29556    3483.9200
25171     347.3800
           ...    
6265        0.0000
54886      67.2750
76820       0.0000
860       244.8600
15795       0.0000
Name: pv_measurement, Length: 73098, dtype: float64

37181      0.0
78736    196.0
64242     -0.0
3431       0.0
30125      0.0
         ...  
89403      0.0
63802     -0.0
44924      0.0
43417      0.0
59210      0.0
Name: pv_measurement, Length: 18275, dtype: float64

# Making model

In [61]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(trainset)
train_data.head()

label = 'pv_measurement'
train_data[label].describe()

predictor = TabularPredictor(label=label, eval_metric='mean_absolute_error')
predictor.fit(train_data, presets='best_quality', time_limit=1000)

No path specified. Models will be saved in: "AutogluonModels/ag-20231112_095621"
Presets specified: ['best_quality']
/Users/oscarmarcussen/anaconda3/envs/ag/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1000s
AutoGluon will save models to "AutogluonModels/ag-20231112_095621"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:41:43 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T6000
Disk Space Avail:   27.93 GB / 494.38 GB (5.6%)
Train Data Rows:    73098
Train Data Columns: 46
Label Col

[1000]	valid_set's l1: 82.458
[1000]	valid_set's l1: 89.0563
[1000]	valid_set's l1: 85.5921
[1000]	valid_set's l1: 88.6204
[1000]	valid_set's l1: 88.3377
[1000]	valid_set's l1: 84.4435


	-85.5507	 = Validation score   (-mean_absolute_error)
	48.56s	 = Training   runtime
	0.45s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 610.38s of the 943.81s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's l1: 77.5427
[2000]	valid_set's l1: 76.1967
[3000]	valid_set's l1: 76.0561
[4000]	valid_set's l1: 75.718
[5000]	valid_set's l1: 75.6579
[6000]	valid_set's l1: 75.5993
[7000]	valid_set's l1: 75.5533
[8000]	valid_set's l1: 75.5315
[9000]	valid_set's l1: 75.5382
[1000]	valid_set's l1: 82.1501
[2000]	valid_set's l1: 80.8184
[3000]	valid_set's l1: 80.5784
[4000]	valid_set's l1: 80.3401
[5000]	valid_set's l1: 80.1145
[6000]	valid_set's l1: 80.0169
[7000]	valid_set's l1: 80.0682
[1000]	valid_set's l1: 79.5795
[2000]	valid_set's l1: 78.567
[3000]	valid_set's l1: 78.0774
[4000]	valid_set's l1: 77.9866
[5000]	valid_set's l1: 77.8332
[6000]	valid_set's l1: 77.8813
[1000]	valid_set's l1: 83.1593
[2000]	valid_set's l1: 81.3458
[3000]	valid_set's l1: 80.7684
[4000]	valid_set's l1: 80.2867
[5000]	valid_set's l1: 80.0202
[6000]	valid_set's l1: 79.9131
[7000]	valid_set's l1: 79.8566


	Ran out of time, early stopping on iteration 7422. Best iteration is:
	[7310]	valid_set's l1: 79.8336


...

	-77.7571	 = Validation score   (-mean_absolute_error)
	381.03s	 = Training   runtime
	4.65s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 222.89s of the 556.32s of remaining time.
	-83.4996	 = Validation score   (-mean_absolute_error)
	62.46s	 = Training   runtime
	1.66s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 157.76s of the 491.2s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Ran out of time, early stopping on iteration 2546.
	Ran out of time, early stopping on iteration 2677.
	Ran out of time, early stopping on iteration 2753.
	Ran out of time, early stopping on iteration 2760.
	Ran out of time, early stopping on iteration 2879.
	Ran out of time, early stopping on iteration 3067.
	Ran out of time, early stopping on iteration 3333.
	Ran out of time, early stopping on iteration 3799.
	-83.0652	 = Validation score   (-mean_absolute_error)
	1

[1000]	valid_set's l1: 77.7665
[1000]	valid_set's l1: 75.2408
[2000]	valid_set's l1: 75.0138
[1000]	valid_set's l1: 80.6767
[2000]	valid_set's l1: 80.493
[3000]	valid_set's l1: 80.3809
[4000]	valid_set's l1: 80.24
[1000]	valid_set's l1: 75.3565
[2000]	valid_set's l1: 74.89
[3000]	valid_set's l1: 74.7275
[4000]	valid_set's l1: 74.4793
[5000]	valid_set's l1: 74.149
[6000]	valid_set's l1: 73.989
[7000]	valid_set's l1: 73.9459
[8000]	valid_set's l1: 73.8104


	Ran out of time, early stopping on iteration 8936. Best iteration is:
	[8802]	valid_set's l1: 73.6847


[1000]	valid_set's l1: 74.4581


	-74.8136	 = Validation score   (-mean_absolute_error)
	155.92s	 = Training   runtime
	1.57s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 174.62s of the 174.61s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-73.1285	 = Validation score   (-mean_absolute_error)
	12.44s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 161.92s of the 161.91s of remaining time.
	-72.0033	 = Validation score   (-mean_absolute_error)
	88.67s	 = Training   runtime
	1.75s	 = Validation runtime
Fitting model: CatBoost_BAG_L2 ... Training model for up to 70.66s of the 70.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-73.3942	 = Validation score   (-mean_absolute_error)
	24.63s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesMSE_BAG_L2 ... Training model fo

# Prediction

In [62]:
test_data = TabularDataset(evalset)

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

37181     0.094130
78736    42.511757
64242    -0.006695
3431      0.146616
30125     0.104305
Name: pv_measurement, dtype: float32

# Evaluation

In [63]:
predictor.evaluate(test_data, silent=True)

{'mean_absolute_error': -69.93463804214322,
 'root_mean_squared_error': -226.7580091159616,
 'mean_squared_error': -51419.194698234525,
 'r2': 0.9129591653817721,
 'pearsonr': 0.9554892197016819,
 'median_absolute_error': -0.7801611423492432}

In [64]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-69.934638,-71.221412,17.334432,19.025172,941.358728,0.002131,0.001255,0.384113,3,True,15
1,ExtraTreesMSE_BAG_L2,-70.200223,-71.627426,13.398209,15.559138,659.306083,0.384265,1.663119,11.817900,2,True,13
2,RandomForestMSE_BAG_L2,-70.205335,-72.003256,13.385929,15.647574,736.160525,0.371985,1.751555,88.672342,2,True,11
3,LightGBM_BAG_L2,-71.364506,-73.128474,13.245606,13.995783,659.930210,0.231662,0.099764,12.442027,2,True,10
4,LightGBMXT_BAG_L2,-71.458866,-74.813603,16.282348,15.470468,803.412730,3.268404,1.574449,155.924547,2,True,9
5,CatBoost_BAG_L2,-72.140883,-73.394220,13.075985,13.935030,672.117800,0.062041,0.039011,24.629617,2,True,12
6,LightGBM_BAG_L1,-73.813384,-77.757115,9.594445,4.652869,381.030287,9.594445,4.652869,381.030287,1,True,4
7,WeightedEnsemble_L2,-74.130029,-75.867801,12.189906,10.763231,647.766604,0.003022,0.000558,0.324865,2,True,8
8,NeuralNetFastAI_BAG_L2,-78.987442,-84.570431,15.359947,14.706247,675.015857,2.346003,0.810228,27.527674,2,True,14
9,CatBoost_BAG_L1,-81.204856,-83.065207,0.152675,0.099136,151.236995,0.152675,0.099136,151.236995,1,True,6


# Predict test data

In [85]:
preds = predictor.predict(testset)
preds = preds.reset_index()
preds = preds.drop(columns=["index"])
preds = preds.rename(columns={"pv_measurement": "prediction"})
preds = preds.reset_index()
preds = preds.rename(columns={"index": "id"})

display(preds)

/Users/oscarmarcussen/anaconda3/envs/ag/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)


91373      3.075854
91374      3.586959
91375      3.929909
91376     71.986893
91377    459.344910
            ...    
93528     81.747849
93529     55.291061
93530     34.913445
93531      9.329406
93532      8.256231
Name: pv_measurement, Length: 2160, dtype: float32

In [91]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission = sample_submission[['id']].merge(preds[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('autogluon_oscar_2.csv', index=False)